# Check (That Tweet) Yo Self 
## Prioritizing Tweets to Fact Check
###### Part 4: Gathering User Data

After exploring the tweets we gathered, we realized it would be hugely beneficial to also have information on the user like number of followers, number of tweets total, etc. In this notebook, we loop through our previously gathered tweets to add the user data. 

In [1]:
import pandas as pd
import numpy as np
from tweetscrape.users_scrape import TweetScrapperUser
import tweetscrape
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

Importing our data

In [2]:
tweet = pd.read_csv('./data/cleaned_with_lysol.csv')

Quick check to see how the user info is returned from Twitter

In [3]:
ts = TweetScrapperUser("PulpNews")
user_info = ts.get_profile_info()

In [4]:
user_info['bio']

'The Fastest Crime News Updates on the Planet!'

This function loops through each user to grab their profile information:

In [5]:
def get_user(df):
    #Start a counter to track progress
    count = 0
    
    #Create our new columns that will eventually hold the user data
    df['user_bio'] = 0
    df['user_location'] = 0
    df['user_url'] = 0
    df['user_tweets'] = 0
    df['user_following'] = 0
    df['user_followers'] = 0
    df['user_favorites'] = 0
    
    #Loop to gather user data and assign it to the correct column
    for user in df['author']:
        count += 1
        try:
            ts = TweetScrapperUser(user)
            user_info = ts.get_profile_info()
            df.loc[df['author'] == user, 'user_bio'] = user_info['bio']
            df.loc[df['author'] == user,'user_location'] = user_info['location']
            df.loc[df['author'] == user,'user_url'] = user_info['url']
            df.loc[df['author'] == user,'user_tweets'] = user_info['tweets']
            df.loc[df['author'] == user,'user_following'] = user_info['following']
            df.loc[df['author'] == user,'user_followers'] = user_info['followers']
            df.loc[df['author'] == user,'user_favorites'] = user_info['favorites']
            if count % 100 == 0:
                print(f'{count} users have been gathered')
                df.to_csv(f'./data/users/{count}_tweet_random.csv', index=False)
        except:
            pass
    
    #Return the final, filled in dataframe
    return df

### Pulling 40k random ones:

We already pulled around 80,000 tweets but going back to grab the user info ix very time consuming. To make it more efficient, we'll take a random set of 40,000 tweets to then grab user info. 

In [9]:
tweet_rand = tweet.sample(n = 40_000, random_state = 21).reset_index(drop=True)

In [10]:
tweet_rand

,id,time,author,author_id,associated_tweet,text,links,hashtags,mentions,reply_count,...,hashtag_count,mention_count,word_count,char_count,link_count,text_sentiment,text_links_removed,clean_text,clean_word_count,clean_char_count
0,1254190074595553281,2020-04-25 16:26:30,Iam_helenna,215204985,1254190074595553281,"Today, we have 1182 cases in Nigeria with 35 d...",[],[''],[''],37,...,0,0,42,270,0,0.4215,"Today, we have 1182 cases in Nigeria with 35 d...",today cases nigeria deaths discharged isolatio...,28,193
1,1253828209075990531,2020-04-24 16:28:34,KerryeHill,2807727004,1253697753479331840,There's no such thing as a medical disinfectan...,[],[''],[''],1,...,0,0,40,248,0,-0.7184,There's no such thing as a medical disinfectan...,thing medical disinfectant use pulmonary syste...,19,152
2,1253460644294283265,2020-04-23 16:08:00,Lmt48430438,1232381432988930049,1253460644294283265,Waiting to see how many people drink disinfect...,[],[''],['@DarcysCartoon'],1,...,0,1,23,130,0,0.0000,Waiting to see how many people drink disinfect...,waiting see many people drink disinfectant tru...,13,93
3,1254194987945865217,2020-04-25 16:46:01,iamshollyyoung,3096323025,1254194987945865217,Today I know there's no result Nigeria can not...,['https://t.co/RRuHGBH1SI'],['#Covid_19'],[''],1,...,1,0,16,118,1,-0.2095,Today I know there's no result Nigeria can not...,today know result nigeria rig imagine ncdc rig...,15,93
4,1253835841685934081,2020-04-24 16:58:54,toddcusuman,588727638,1253835841685934081,New York rapper Fred the Godson dies at 35 aft...,['https://t.co/rXOi5YEoZl'],[''],[''],0,...,0,0,14,122,1,0.0000,New York rapper Fred the Godson dies at 35 aft...,new york rapper fred godson dies coronavirus h...,17,93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,1254190054295113729,2020-04-25 16:26:25,StevieK15,400491014,1254172638307586049,This is why you shouldn’t use hydroxychloroqui...,[],[''],[''],0,...,0,0,22,131,0,0.4019,This is why you shouldn’t use hydroxychloroqui...,use hydroxychloroquine iv disinfectant uv ther...,11,80
39996,1254193792221097985,2020-04-25 16:41:16,JasonJaytay1969,3053529784,1254093268884893696,It's the craziness of self isolation!! LOLTher...,[],[''],[''],0,...,0,0,25,137,0,-0.4559,It's the craziness of self isolation!! LOLTher...,craziness self isolation lolthere one mikey kn...,10,66
39997,1253823959319142402,2020-04-24 16:11:41,kILtedbreeder,349449067,1253823959319142402,Day 42 of social isolation...that’s two days l...,['https://t.co/rq3x6Jv58h'],[''],"['@disneyplus', '@DisneyParks']",1,...,0,2,41,262,1,-0.4215,Day 42 of social isolation...that’s two days l...,day social isolation two days longer biblical ...,28,179
39998,1254194918765068288,2020-04-25 16:45:45,HadejiaArc,1214683184484560896,1253329154415529985,Beside you said I have to go to the bank and y...,[],[''],[''],0,...,0,0,48,225,0,0.7158,Beside you said I have to go to the bank and y...,beside said go bank know operating due covid t...,23,137


As mentioned above, this was extremely time consuming. While we had hopped for 40k, grabbing just over 30k took almost 12 hours. Hence why the cell below is unrun, after restarting the notebook we didn't want to wait another 12 hours for it to complete.

In [ ]:
tweet_rand = get_user(tweet_rand)
tweet_rand.to_csv('./data/40k_random.csv', index = False)

Unable to get last tweet timestamp


100 users have been gathered


Unable to get last tweet timestamp
Unable to get last tweet timestamp


200 users have been gathered


Unable to get last tweet timestamp


...

Unable to get last tweet timestamp
Unable to get last tweet timestamp


32400 users have been gathered
32500 users have been gathered
32600 users have been gathered
32700 users have been gathered
32800 users have been gathered
32900 users have been gathered
33000 users have been gathered


Unable to get last tweet timestamp
Unable to get last tweet timestamp


33100 users have been gathered
33200 users have been gathered


Unable to get last tweet timestamp


Now that we have user data, we'll do some more EDA then get to modeling.